In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests; from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [7]:
gu_list = ["마포구","서대문구","은평구","종로구","중구","용산구","성동구","광진구",
           "동대문구","성북구","강북구","도봉구","노원구","중랑구","강동구","송파구",
           "강남구","서초구","관악구","동작구","영등포구","금천구","구로구","양천구","강서구"]


In [13]:
gu = "구로구"

In [14]:
keyword = "애견카페"

In [18]:
driver = webdriver.Chrome(executable_path= "chromedriver.exe")
url = "https://map.kakao.com/"
driver.get(url)
action = ActionChains(driver)
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
search_area.send_keys(gu + " " + keyword)  # 검색어 입력
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
driver.implicitly_wait(3)
more_page = driver.find_element_by_id("info.search.place.more")
print(more_page.get_attribute("class"))
if "HIDDEN" in more_page.get_attribute("class").split(" "):
    PAGE = 1
else:
    

more HIDDEN


In [19]:
driver = webdriver.Chrome(executable_path= "chromedriver.exe")
url = "https://map.kakao.com/"
driver.get(url)
action = ActionChains(driver)
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
search_area.send_keys(find + ' 애견동반카페')  # 검색어 입력
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
driver.implicitly_wait(3) # 기다려 주자
more_page = driver.find_element_by_id("info.search.place.more")
more_page.send_keys(Keys.ENTER) # 더보기 누르고
time.sleep(1)
# next 사용 가능?
next_btn = driver.find_element_by_id("info.search.page.next")
has_next = "disabled" not in next_btn.get_attribute("class").split(" ")


In [20]:
has_next

False

In [21]:
next_btn.get_attribute("class")

'next disabled'

In [31]:
def address_to_latitude(address):
    global lat
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    print(url)
    result = requests.get(urlparse(url).geturl(),
                     headers = {"Authorization": "KakaoAK " + "889c9d9bd1489eb337cfeb167093b9df"}) # Rest API
    print(result)
    json_obj = result.json()
    print(json_obj['documents'])
    for document in json_obj['documents']:
        lat = document['y']
    return lat

def address_to_longtitude(address):
    global long
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    print(url)
    result = requests.get(urlparse(url).geturl(),
                     headers = {"Authorization": "KakaoAK "  + "889c9d9bd1489eb337cfeb167093b9df"})
    print(result)
    json_obj = result.json()
    print(json_obj['documents'])
    for document in json_obj['documents']:
         long = document['x']
    return long

In [27]:
results = []

In [32]:
Page=1
while True: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
    time.sleep(1)
    # place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li:nth-child(1)')
    # 페이지 루프
    #info\.search\.page\.no1 ~ .no5
    page_links = driver.find_elements_by_css_selector("#info\.search\.page a")
    pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
    print(len(pages), "개의 페이지 있음")
    # pages를 하나씩 클릭하면서
    for i in range(1, 6):
        xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
        try:
            page = driver.find_element_by_xpath(xPath)
            page.send_keys(Keys.ENTER)
        except ElementNotInteractableException:
            print('End of Page')
            break;
        time.sleep(3)
        place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
        for p in place_lists: # WebElement
            # print(p.get_attribute('innerHTML'))
            # print("type of p:", type(p))
            store_html = p.get_attribute('innerHTML')
            store_info = BeautifulSoup(store_html, "html.parser")
            # BS -> 분석
            #
            place_name = store_info.select('.head_item > .tit_name > .link_name')
            
            if len(place_name) == 0:
                continue # 광고
            name = store_info.select('.head_item > .tit_name > .link_name')[0].text
            address = store_info.select('.info_item > .addr > p')[0].text
            print(address)
            latitude = address_to_latitude(address)
            longtitude = address_to_longtitude(address)
            results.append((name, address, latitude, longtitude))
            
        print(i, ' of', ' [ ' , Page, ' ] ')
    next_btn = driver.find_element_by_id("info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    if not has_next:
        print('Arrow is Disabled')
        driver.close()
        break # 다음 페이지 없으니까 종료
    else: # 다음 페이지 있으면
        Page += 1
        next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")
    

3 개의 페이지 있음
서울 서대문구 연희로28길 3
https://dapi.kakao.com/v2/local/search/address.json?query=서울 서대문구 연희로28길 3
<Response [200]>
[{'address': {'address_name': '서울 서대문구 연희동 138-46', 'b_code': '1141011700', 'h_code': '1141061500', 'main_address_no': '138', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '서대문구', 'region_3depth_h_name': '연희동', 'region_3depth_name': '연희동', 'sub_address_no': '46', 'x': '126.9358529766', 'y': '37.5746197032227'}, 'address_name': '서울 서대문구 연희로28길 3', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 서대문구 연희로28길 3', 'building_name': '', 'main_building_no': '3', 'region_1depth_name': '서울', 'region_2depth_name': '서대문구', 'region_3depth_name': '연희동', 'road_name': '연희로28길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.9358529766', 'y': '37.5746197032227', 'zone_no': '03720'}, 'x': '126.9358529766', 'y': '37.5746197032227'}]
https://dapi.kakao.com/v2/local/search/address.json?query=서울 서대문구 연희로28길 3
<Response [200]>
[{'address':

<Response [200]>
[{'address': {'address_name': '서울 은평구 진관동 100', 'b_code': '1138011400', 'h_code': '1138069000', 'main_address_no': '100', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '은평구', 'region_3depth_h_name': '진관동', 'region_3depth_name': '진관동', 'sub_address_no': '', 'x': '126.917695482942', 'y': '37.6341922549283'}, 'address_name': '서울 은평구 통일로 1030', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 은평구 통일로 1030', 'building_name': '은평헤스티아', 'main_building_no': '1030', 'region_1depth_name': '서울', 'region_2depth_name': '은평구', 'region_3depth_name': '진관동', 'road_name': '통일로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.917695482942', 'y': '37.6341922549283', 'zone_no': '03311'}, 'x': '126.917695482942', 'y': '37.6341922549283'}]
서울 마포구 와우산로17길 19-5
https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 와우산로17길 19-5
<Response [200]>
[{'address': {'address_name': '서울 마포구 서교동 409-18', 'b_code': '1144012000', 'h_code': '11440

<Response [200]>
[{'address': {'address_name': '서울 마포구 용강동 494-127', 'b_code': '1144010500', 'h_code': '1144059000', 'main_address_no': '494', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '마포구', 'region_3depth_h_name': '용강동', 'region_3depth_name': '용강동', 'sub_address_no': '127', 'x': '126.938709239347', 'y': '37.5429118024564'}, 'address_name': '서울 마포구 대흥로6길 7', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 마포구 대흥로6길 7', 'building_name': '리버빌', 'main_building_no': '7', 'region_1depth_name': '서울', 'region_2depth_name': '마포구', 'region_3depth_name': '용강동', 'road_name': '대흥로6길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.938709239347', 'y': '37.5429118024564', 'zone_no': '04162'}, 'x': '126.938709239347', 'y': '37.5429118024564'}]
https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 대흥로6길 7
<Response [200]>
[{'address': {'address_name': '서울 마포구 용강동 494-127', 'b_code': '1144010500', 'h_code': '1144059000', 'main_address_n

서울 양천구 목동중앙남로9길 41 1층 101,102호
https://dapi.kakao.com/v2/local/search/address.json?query=서울 양천구 목동중앙남로9길 41 1층 101,102호
<Response [200]>
[{'address': {'address_name': '서울 양천구 목동 730-18', 'b_code': '1147010200', 'h_code': '1147054000', 'main_address_no': '730', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '양천구', 'region_3depth_h_name': '목4동', 'region_3depth_name': '목동', 'sub_address_no': '18', 'x': '126.86414671847', 'y': '37.5378713602665'}, 'address_name': '서울 양천구 목동중앙남로9길 41', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 양천구 목동중앙남로9길 41', 'building_name': '', 'main_building_no': '41', 'region_1depth_name': '서울', 'region_2depth_name': '양천구', 'region_3depth_name': '목동', 'road_name': '목동중앙남로9길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.86414671847', 'y': '37.5378713602665', 'zone_no': '07959'}, 'x': '126.86414671847', 'y': '37.5378713602665'}]
https://dapi.kakao.com/v2/local/search/address.json?query=서울 양천구 목동중앙남로9길 41 1층 101,

서울 강서구 강서로15길 18 1층
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 강서로15길 18 1층
<Response [200]>
[{'address': {'address_name': '서울 강서구 화곡동 348-17', 'b_code': '1150010300', 'h_code': '1150054000', 'main_address_no': '348', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강서구', 'region_3depth_h_name': '화곡1동', 'region_3depth_name': '화곡동', 'sub_address_no': '17', 'x': '126.844975557473', 'y': '37.5325330214338'}, 'address_name': '서울 강서구 강서로15길 18', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 강서구 강서로15길 18', 'building_name': '', 'main_building_no': '18', 'region_1depth_name': '서울', 'region_2depth_name': '강서구', 'region_3depth_name': '화곡동', 'road_name': '강서로15길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.844975557473', 'y': '37.5325330214338', 'zone_no': '07770'}, 'x': '126.844975557473', 'y': '37.5325330214338'}]
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 강서로15길 18 1층
<Response [200]>
[{'address

<Response [200]>
[{'address': {'address_name': '서울 용산구 한강로2가 143-1', 'b_code': '1117012500', 'h_code': '1117062500', 'main_address_no': '143', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '용산구', 'region_3depth_h_name': '한강로동', 'region_3depth_name': '한강로2가', 'sub_address_no': '1', 'x': '126.969969527032', 'y': '37.5301466215658'}, 'address_name': '서울 용산구 한강대로40길 14', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 용산구 한강대로40길 14', 'building_name': '', 'main_building_no': '14', 'region_1depth_name': '서울', 'region_2depth_name': '용산구', 'region_3depth_name': '한강로2가', 'road_name': '한강대로40길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.969969527032', 'y': '37.5301466215658', 'zone_no': '04386'}, 'x': '126.969969527032', 'y': '37.5301466215658'}]
https://dapi.kakao.com/v2/local/search/address.json?query=서울 용산구 한강대로40길 14 지하1층, 1, 2층
<Response [200]>
[{'address': {'address_name': '서울 용산구 한강로2가 143-1', 'b_code': '1117012500', 'h_code': '1117

In [33]:
results

[('카페로뎀', '서울 서대문구 연희로28길 3', '37.5746197032227', '126.9358529766'),
 ('카페프롬나즈데떼',
  '서울 양천구 오목로 300 206동상가 1층',
  '37.5248304019543',
  '126.87002254331'),
 ('카페미우', '서울 용산구 새창로14길 7 1층', '37.539184966822', '126.958516265438'),
 ('카페별뜨락', '서울 종로구 창의문로 119', '37.592096942441', '126.96706863952'),
 ('반하다카페', '서울 종로구 자하문로 35-3', '37.5790436149798', '126.97115806662'),
 ('카페 바로',
  '서울 은평구 통일로 1030 은평 헤스티아 B102호',
  '37.6341922549283',
  '126.917695482942'),
 ('겨울이머무는집', '서울 마포구 와우산로17길 19-5', '37.5499573093862', '126.921864513339'),
 ('그냥방실',
  '서울 마포구 희우정로20길 71 와와모빌딩 1층 102호',
  '37.556259678373',
  '126.904220131871'),
 ('할리스 공덕경의선숲길점',
  '서울 마포구 백범로28길 22 일양빌딩 신관 1층',
  '37.544603767606',
  '126.946646153695'),
 ('55라이트', '서울 마포구 희우정로 58 1층', '37.5517223827873', '126.90560986618'),
 ('나만믿개', '서울 마포구 성암로15길 34 2층', '37.5773467650035', '126.896129432676'),
 ('쿄로쿄로', '서울 마포구 대흥로6길 7', '37.5429118024564', '126.938709239347'),
 ('모카하라', '서울 마포구 희우정로 46-15', '37.5516127976763', '126.906827